# Nutrient Balance - Phosphorus and Nitrogen
Phosphorus and nitrogen balance (surfeit +, deficit -) time series by country/territory.  

Note that this is a parameterized widget; the specification passed to the API will not be renderable without the geostore identifier being inserted.  

_Author: Peter Kerins_  
_Created: 23 Aug 2021_  
_Environment: jupyterlab_  

## Style
- Vega chart
- Ocean Watch country page - parameterized chart
- Line chart
- Multiline

## Data
Underlying dataset: [foo.063.rw0 Nutrient Balance](https://resourcewatch.carto.com/u/wri-rw/dataset/foo_063_rw0_nutrient_balance_edit)  
Widget data: [ow_widget_territory](https://resourcewatch.carto.com/u/wri-rw/dataset/ow_widget_territory) (master table for all territory-based widgets on Ocean Watch)  

## Preparation

In [1]:
import json
from vega import Vega
from IPython.display import display

In [2]:
def Vega(spec):
    bundle = {}
    bundle['application/vnd.vega.v5+json'] = spec
    display(bundle, raw=True)

In [3]:
widget_width = 400
widget_height = 300

# Widget

## Demo Query
`gid_0 = <territory>` used as stand-in for parameterization
```sql
SELECT gid_0, name_0, variable, date, value 
FROM ow_widget_territory
WHERE widget ILIKE 'nutrient_balance'
AND gid_0 = 'BRA' 
ORDER BY gid_0 ASC, variable ASC, date ASC
```

## Parameterized Query
```sql
SELECT territory.gid_0, territory.name_0, variable, date, value
FROM ow_widget_territory AS territory
LEFT OUTER JOIN gadm36_0 AS gadm ON territory.gid_0 = gadm.gid_0
WHERE territory.widget ILIKE 'nutrient_balance'
AND gadm.{{geostore_env}}='{{geostore_id}}'
ORDER BY territory.gid_0 ASC, territory.variable ASC, territory.date ASC
```

## Specification
Vega code presumes RW-style `config` element present

In [5]:
spec=json.loads("""
{
  "schema": "https://vega.github.io/schema/vega/v5.json",
  "description": "Nutrient balance of phosphorus and nitrogen time series, by country",
  "width": 400,
  "height": 250,
  "padding": 0,
  "autosize": {"type": "fit", "contains": "padding"},
  "signals": [],
  "data": [
    {
      "name": "table",
      "url": "https://wri-rw.carto.com/api/v2/sql?q=SELECT gid_0, name_0, variable, date, value FROM ow_widget_territory WHERE widget ILIKE 'nutrient_balance' AND gid_0 = 'ARG' ORDER BY gid_0 ASC, variable ASC, date ASC",
      "format": {
        "type": "json",
        "property": "rows",
        "parse": {"date": "utc:'%Y'"}
      },
      "transform": [
        {
          "type": "formula",
          "expr": "slice(datum.variable,0,indexof(datum.variable,' '))",
          "as": "nutrient"
        }
      ]
    }
  ],
  "scales": [
    {
      "name": "xscale",
      "type": "time",
      "domain": {"data": "table", "fields": ["date"]},
      "range": "width",
      "padding": 0
    },
    {
      "name": "yscale",
      "type": "linear",
      "domain": {"data": "table", "field": "value"},
      "range": "height",
      "padding": 0
    },
    {
      "name": "colorscale",
      "type": "ordinal",
      "domain": [
        "Nitrogen",
        "Phosphorus"
      ],
      "range": [
        "blue",
        "purple"
      ]
    },
    {
      "name": "shapescale",
      "type": "ordinal",
      "domain": [
        "Nitrogen",
        "Phosphorus"
      ],
      "range": [
        "circle",
        "diamond"
      ]
    }
  ],
  "axes": [
    {
      "orient": "bottom",
      "scale": "xscale",
      "labelFlush": true,
      "labelOverlap": true,
      "labelBaseline": "middle",
      "domain": true,
      "ticks": true,
      "tickSize": 5,
      "grid": false,
      "labelFontSize": 10,
      "labelFont": "Arial",
      "format": "%Y"
    },
    {
      "orient": "left",
      "scale": "yscale",
      "title": "Nutrient balance (kg/ha)",
      "labelBaseline": "middle",
      "labelAlign": "right",
      "labelPadding": 5,
      "domain": false,
      "grid": true,
      "gridOpacity": 0.2,
      "labelFontSize": 10,
      "offset": 3,
      "ticks": false,
      "labelFont": "Arial",
      "encode": {
        "ticks": {
          "update": {
            "stroke": {"value": "steelblue"}
          }
        },
        "labels": {
          "interactive": true,
          "update": {
            "fill": {"value": "grey"}
          },
          "hover": {
            "fill": {"value": "firebrick"}
          }
        }
      }
    }
  ],
  "marks": [
    {
      "type": "group",
      "from": {
        "facet": {
          "name": "series",
          "data": "table",
          "groupby": "nutrient"
        }
      },
      "marks": [
        {
          "type": "line",
          "from": {"data": "series"},
          "encode": {
            "enter": {
              "x": {"scale": "xscale", "field": "date"},
              "y": {"scale": "yscale", "field": "value"},
              "stroke": {"scale": "colorscale", "field": "nutrient"},
              "strokeWidth": {"value": 1.5},
              "strokeDash": {"value": "2,1"},
              "interpolate": {"value": "linear"}
            }
          }
        },
        {
          "type": "symbol",
          "from": {"data": "series"},
          "encode": {
            "enter": {
              "x": {"scale": "xscale", "field": "date"},
              "y": {"scale": "yscale", "field": "value"},
              "stroke": {"scale": "colorscale", "field": "nutrient"},
              "shape": {"scale": "shapescale", "field": "nutrient"},
              "strokeWidth": {"value": 1.5},
              "size": {"signal": "width/4"},
              "tooltip": {
                "signal": "{'Year':timeFormat(datum.date,'%Y'),'Nutrient':datum.nutrient,'Balance':(datum.value >= 0 ? '+' : '')+format(datum.value,'')+' kg/ha'}"
              },
              "interpolate": {"value": "monotone"},
              "fillOpacity": {"value": 1.0},
              "strokeOpacity": {"value": 1.0}
            },
            "update": {
              "fill": {"value": "white"}
            },
            "hover": {
              "fill": {"scale": "colorscale", "field": "nutrient"}
            }
          }
        }
      ]
    }
  ]
}
""")
vega_view=dict(spec)
vega_view['width']=widget_width
vega_view['height']=widget_height
Vega(vega_view)

# RW API

Parameterized widgets created via [widget scripts](https://github.com/resource-watch/data-team-tools/tree/master/advanced_widget_writer).  

[foo.063.rw0 Nutrient Balance]() ([Admin](https://resourcewatch.org/admin/data/datasets/adba3cb5-7f18-491f-b31a-1b8b8c03fa67/edit))  
`adba3cb5-7f18-491f-b31a-1b8b8c03fa67`  
[Dataset](https://api.resourcewatch.org/v1/dataset/adba3cb5-7f18-491f-b31a-1b8b8c03fa67/), [Widgets](https://api.resourcewatch.org/v1/dataset/adba3cb5-7f18-491f-b31a-1b8b8c03fa67/widget)

In [5]:
dataset_id = 'adba3cb5-7f18-491f-b31a-1b8b8c03fa67'
name = 'Nutrient Balance'
description = 'Time series of nutrient balance for phosphorus and nitrogen, by country'

- [Country](https://api.resourcewatch.org/v1/widget/4e8f077f-bed7-44f7-8d52-d793b485f301?env=production) `4e8f077f-bed7-44f7-8d52-d793b485f301`  